In [2]:
# Import functions for graphing and methods (also includes some libraries needed for data analysis)
from graphing_funcs import *
from method_funcs import *
from daymet_funcs import *

from itertools import combinations
from random import sample, seed

%matplotlib inline

# set style parameters for graphs
sns.set_style("darkgrid", {'axes.edgecolor': 'black'})
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["legend.edgecolor"] = 'black'
plt.rcParams["legend.fontsize"] = 13
plt.rcParams['figure.dpi'] = 300

# Import libraries for sql connection
import mysql.connector
import pickle
import warnings

warnings.filterwarnings('ignore')

# Load credentials for login (hidden and not added to repo)
with open("login_cred.pkl", "rb") as fp:
    config = pickle.load(fp)
    
cnx = mysql.connector.connect(**config) # connection point

In [ ]:
def eval_metrics(val_data, temp_b, delta_b):
    '''
    Calculate number of flags being set for the temperature bounds and delta step based on 
    the validation data. This is only for a single year at a time, working by finding which
    dates occur in the validation data and pulling them from the estimates (temp_b & delta_b).
    
    Returns number of flags for upper and lower bound, for both overall temperature and the 
    delta values.
    '''
    # -------------------- Temperature bounds flag calculation --------------------
    v_idx = [x.split('-', 1)[1] for x in val_data.index.values] # get all month_day occurances
    est_val = temp_b.loc[v_idx] # subset estimates with only val_data dates

    vals = val_data.values # get all temp readings (2d array)

    # Get lower bound estimates (reshape as 2d array column)
    temp_lb = np.reshape(est_val.daily_min.values, (-1, 1))

    # Get upper bound estimates (reshape as 2d array column)
    temp_ub = np.reshape(est_val.daily_max.values, (-1, 1))

    # Calculate number of flags being set on validation data
    temp_lb_flags = sum((vals < temp_lb).flatten())
    temp_ub_flags = sum((vals > temp_ub).flatten())

    # -------------------- Delta step flag calculation --------------------
    s_tmp_v = val_data.copy() # create copy (don't alter original)
    s_tmp_v['month'] = [int(x.split('-')[1]) for x in s_tmp_v.index.values] # create month column

    delta_lb_flags = 0 # count number of lower bound flags
    delta_ub_flags = 0 # count number of upper bound flags

    for i in s_tmp_v.month.unique(): # iterate over each month
        v = s_tmp_v[s_tmp_v.month == i] # subset single month
        v = v.drop(columns=['month']) # remove month column
        deltas = np.diff(v.values.flatten()) # calculate deltas
        b = delta_b.iloc[i-1,:].values # pull bounds
        delta_lb_flags += sum(deltas < b[0]) # increment lower bound flags
        delta_ub_flags += sum(deltas > b[1]) # increment upper bound flags

    # Return calculated flags
    return temp_lb_flags, temp_ub_flags, delta_lb_flags, delta_ub_flags


def perform_cross_val(dates, year_idx, s_pivot, n_folds, n_years):
    '''
    Function will perform cross validation on a given station, with provided information on dates, 
    year index dictionary, and the pivot data table. The average is returned for each subset of 
    years that are used for estimation data. 
    '''
    seed(36) # set random seed

    # Dataframe to hold results from cross validation
    res = pd.DataFrame(columns=['Estimation Years',
                                'TEMP_LB_FLAG', 'TEMP_UB_FLAG', 'DELTA_LB_FLAG', 'DELTA_UB_FLAG'])

    for i in range(1, n_years+1): # iterate over different number of years to use for estimation
        # Get random (repeatable) sample of n_folds of random combinations (estimation years)
        year_combos = sample(list(combinations(dates, i)), n_folds)

        # Define variables to hold number of flags being sets
        delta_lb_avg = 0
        delta_ub_avg = 0
        temp_lb_avg = 0
        temp_ub_avg = 0

        for j in range(n_folds): # iterate over each fold (total of n_folds)

            e = year_combos[j] # get estimation years (array of lists - subset by j)
            v = [2023] # list(set(e) ^ set(dates)) # get validation years

            # Get all estimation and validation data index values
            # np.r_ turns index into range (i.e. 34:45 turns into [34, 35, ..., 44, 45])
            # flatten into single array and use to subset data frame
            e_idx = np.concatenate([np.r_[year_idx[x][0]:year_idx[x][1]] for x in e])
            v_idx = np.concatenate([np.r_[year_idx[x][0]:year_idx[x][1]] for x in v])

            # Split data (estimation and validation sets)
            s_pivot_e = s_pivot.iloc[e_idx, :]
            s_pivot_v = s_pivot.iloc[v_idx, :]

            # Get daily estimates of min/max (with filter applied)
            temp_bounds = temp_bounds_modified(s_pivot_e, 0.99, 15, 0.8, True)

            # Get monthly estimates of min/max (with filter applied)
            delta_bounds = get_delta_est(s_pivot_e, 0.9, 1)

            # Calculate number of flags being set
            temp_l, temp_u, delta_l, delta_u = eval_metrics(s_pivot_v, temp_bounds, delta_bounds)

            # Increment running average of number of flags
            delta_lb_avg += delta_l
            delta_ub_avg += delta_u
            temp_lb_avg += temp_l
            temp_ub_avg += temp_u

        # Divide running average of number of flags
        delta_lb_avg /= n_folds
        delta_ub_avg /= n_folds
        temp_lb_avg /= n_folds
        temp_ub_avg /= n_folds

        # Add new row to data frame
        res.loc[len(res)] = [len(e), temp_lb_avg, temp_ub_avg, delta_lb_avg, delta_ub_avg]
        
    res = res.set_index("Estimation Years")
        
    return res 

In [37]:
for station in [115]:
    
    s_pivot, years = create_pivot(station) # Create pivot table for station
    years.insert(0,0) # insert start date
    years.append(len(s_pivot))
    
    # Find which dates are needed for data 
    # (+1 since we add index where final date occurs, but may not be full year)
    dates = np.arange(2024-len(years)+1,2024)

    year_idx = {} # dictionary to hold year & index values

    for i, y in enumerate(dates): # create dictionary + index pairings
        year_idx[y] = [years[i], years[i+1]]
        
    dates = np.delete(dates, -1) # remove 2023 (always validation)

    # Run cross validation on given station (final param is number of years to use in estimation)
    # Second to last parameter is number of folds (i.e. different combos) for given years to use
    # NOTE: the max is 8, since 1 year needs to be held out for validation (2023)
    res_df = perform_cross_val(dates, year_idx, s_pivot, 5, 8)

In [38]:
res_df

,TEMP_LB_FLAG,TEMP_UB_FLAG,DELTA_LB_FLAG,DELTA_UB_FLAG
Estimation Years,,,,
1.0,162.8,350.8,19.4,16.2
2.0,33.2,41.8,14.2,14.0
3.0,10.0,18.4,18.0,16.2
4.0,0.0,3.0,17.8,16.4
5.0,0.0,2.2,17.0,16.4
6.0,0.0,0.0,16.0,15.6
7.0,0.0,0.0,15.0,15.6
8.0,0.0,0.0,15.8,15.0
